# N. Load libraries

In [1]:
import time, torch
from model_arch import QPPNet
from dataset.terrier_tpch_dataset.terrier_utils import TerrierTPCHDataSet
from dataset.postgres_tpch_dataset.tpch_utils import PSQLTPCHDataSet
from dataset.oltp_dataset.oltp_utils import OLTPDataSet
import argparse

import os

# I. Load Data

## 1.1 Generate Processed Plans for QPPNet based on the original plans

In [5]:
# generate the query workload
initial_plan_dir = '../pg_optimizer_experiment_utils/papers/Cost_Model/TPC-H/lcm/sf=1/parallelism=4/json_plans/'

# load plan from the dir and aggregate them into a list
fnames = [fname for fname in os.listdir(initial_plan_dir) if fname.endswith('0')]
fnames = sorted(fnames, key=lambda fname: int(fname.split('query_')[1][:-7]))
# for fname in fnames:
#     with open(os.path.join(initial_plan_dir, fname), 'r') as f:
#         lines = f.readlines()
#         plan = ''.join(lines[2:len(lines)-2])
#         plan_parts = plan.split('+')
#         plan = ''.join([part.strip() for part in plan_parts])
#         initial_plans.append(plan)
output_filepath_template = './dataset/lcm_tpch/sf=1/query_{}_plans.json'
template_plans = []
for i in range(len(fnames)):
    # load plans
    with open(os.path.join(initial_plan_dir, fnames[i]), 'r') as f:
        lines = f.readlines()
        plan = ''.join(lines[2:len(lines)-2])
        plan_parts = plan.split('+')
        plan = ''.join([part.strip() for part in plan_parts])
        template_plans.append(plan)
    
    # write processed plans to file
    if (i+1) % 500 == 0:
        with open(output_filepath_template.format(int(i/500+1)), 'w') as f:
            for plan in template_plans:
                f.write(plan + '\n')
        template_plans = []

## 1.2 Prepare TPC-H dataset for QPPNet

In [13]:
class Option:
    def __init__(self):
        # options used to prepare data
        self.batch_size = 32
        self.data_dir = './dataset/lcm_tpch/sf=1/'
        self.test_time = False
        self.mean_range_dict= 'mean_range_dict.pickle'

        # options used to prepare model
        self.dataset = 'PSQLTPCH'
        self.SGD = False
        self.lr = 0.001
        self.scheduler = True
        self.step_size = 1000
        self.gamma = 0.95
        self.start_epoch = 0
        self.end_epoch = 250000
        self.test_time = False
        self.save_dir = './saved_model'

        # options used during training
        self.logfile = 'train_loss.txt'
        self.save_latest_epoch_freq = 1000
opt = Option()

In [3]:
dataset = PSQLTPCHDataSet(opt)

Number of groups per query:  [1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]
{'Aggregate': (array([7.6170731e+01, 2.8366740e+04, 8.5155617e+04, 3.4146342e-01,
       4.1463414e-01, 2.4390244e-01, 1.0000000e+00], dtype=float32), array([2.8000000e+02, 4.7512000e+05, 3.5309031e+05, 1.0000001e+00,
       1.0000001e+00, 1.0000001e+00, 1.0000001e+00], dtype=float32)), 'Gather Merge': (array([  110.1  ,  6631.639, 62396.   ], dtype=float32), array([   272.  ,  62947.  , 184345.12], dtype=float32)), 'Sort': (array([1.0572727e+02, 4.2553672e+04, 9.0037484e+04, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 4.5454547e-02,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.3636364e-01, 1.3636364e-01, 0.0000000e+00, 0.0000000e+00,
      

In [4]:
print("dataset_size", dataset.datasize)

dataset_size 8800


# II. Initialize QPPNet

In [5]:
torch.set_default_tensor_type(torch.FloatTensor)

In [14]:
qpp = QPPNet(opt)

# III. Train QPPNet

In [7]:
def save_opt(opt, logf):
    """Print and save options
    It will print both current options and default values(if different).
    It will save options into a text file / [checkpoints_dir] / opt.txt
    """
    message = ''
    message += '----------------- Options ---------------\n'
    for k, v in sorted(vars(opt).items()):
        comment = ''
        # default = parser.get_default(k)
        # if v != default:
        #     comment = '\t[default: %s]' % str(default)
        message += '{:>25}: {:<30}{}\n'.format(str(k), str(v), comment)
    message += '----------------- End -------------------'
    print(message)
    logf.write(message)
    logf.write('\n')

In [12]:
logf = open(opt.logfile, 'w+')
save_opt(opt, logf)
#qpp.test_dataset = dataset.create_test_data(opt)
qpp.test_dataset = dataset.test_dataset

total_iter = 0
for epoch in range(opt.start_epoch, opt.end_epoch):
# for epoch in range(0, opt.end_epoch):
    epoch_start_time = time.time()  # timer for entire epoch
    iter_data_time = time.time()    # timer for data loading per iteration
    epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch

    samp_dicts = dataset.sample_data()
    total_iter += opt.batch_size

    qpp.set_input(samp_dicts)
    qpp.optimize_parameters(epoch)
    logf.write("epoch: " + str(epoch) + "; iter_num: " + str(total_iter) \
                + '; total_loss: {}; test_loss: {}; pred_err: {}; R(q): {}' \
                .format(qpp.last_total_loss, qpp.last_test_loss,
                        qpp.last_pred_err, qpp.last_rq))

    #if total_iters % opt.print_freq == 0:    # print training losses and save logging information to the disk
    losses = qpp.get_current_losses()
    loss_str = "losses: "
    for op in losses:
        loss_str += str(op) + " [" + str(losses[op]) + "]; "

    if epoch % 50 == 0:
        print("epoch: " + str(epoch) + "; iter_num: " + str(total_iter) \
                + '; total_loss: {}; test_loss: {}; pred_err: {}; R(q): {}' \
                .format(qpp.last_total_loss, qpp.last_test_loss,
                        qpp.last_pred_err, qpp.last_rq))
        print(loss_str)


    logf.write(loss_str + '\n')

    if (epoch + 1) % opt.save_latest_epoch_freq == 0:   # cache our latest model every <save_latest_freq> iterations
        print('saving the latest model (epoch %d, total_iters %d)' % (epoch + 1, total_iter))
        qpp.save_units(epoch + 1)

logf.close()

feat_vec [[ 0.10811689 -0.08951143 -0.1714581   0.          0.          0.
   0.         -0.04545454  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.86363626  0.86363626  0.          0.          0.         -0.04545454
   0.          0.          0.         -0.04545454  0.          0.
  -0.13636363  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.         -0.04545454  0.          0.
  -0.04545454 -0.18181817 -0.04545454  0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.         -0.04545454  0.          0.         -0.09090909 -0.09090909
  -0.09090909  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.         -0.04545454
  -0.04545454 -0.04545454 -0.04545454  0.  

NameError: name 'exit' is not defined